# Исследование надежности заемщиков


Описание проекта: 

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.



## Откройте таблицу и изучите общую информацию о данных

In [3]:
import pandas as pd
import seaborn as sns
import statistics as st

try:
    data = pd.read_csv('/content/sample_data/01_data preprocessing.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Задание 3. Выведите основную информацию о датафрейме с помощью метода `info()`.**

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [ ]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце вы обработаете на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработайте значения в этом столбце: замените все отрицательные значения положительными с помощью метода `abs()`.

In [ ]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведите медианное значение трудового стажа `days_employed` в днях.

In [ ]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

In [ ]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [ ]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [ ]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

Заполните пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [ ]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Замените вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.

In [ ]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработайте неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к нижнему регистру. Проверьте остальные столбцы.

In [ ]:
data['education'] = data['education'].str.lower()

In [ ]:
data.duplicated().sum()

71

In [ ]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создайте в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используйте собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [ ]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [ ]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведите на экран перечень уникальных целей взятия кредита из столбца `purpose`.

In [ ]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создайте функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используйте собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучите данные в столбце `purpose` и определите, какие подстроки помогут вам правильно определить категорию.**

In [ ]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

Краткое описание методики решения: 
* Определим отношение (процент) погашенных в срок кредитов к общему числу кредитов для каждой категории (без детей, 1 ребенок, 2 ребенка и тд).
* Построим график (гистограмму), на котором по оси Y укажем параметр 'вероятность возврата кредита', а по оси Х - категорию (без детей, и тд), или соберем данные в сводную таблицу, проведя сортировку по убыванию или возрастанию, н
* Опираясь на результаты - сфорируем выводы

Начнем с группировки данных по признаку 'children' и выполним необходимые операции для расчета искомой вероятности возврата кредита без задолженности (столбец 'without_debt')

In [ ]:
data_grouped_children = data.groupby('children').agg({'debt' : ['count', 'sum']}) # вгруппируем и применяем функции count and sum
data_grouped_children['with_debt'] = data_grouped_children['debt']['sum'] / data_grouped_children['debt']['count'] # вероятность вернуть кредит с задержкой
data_grouped_children['without_debt'] = 1 - data_grouped_children['with_debt'] # вероятность вернуть кредит без задержкой (находим как вероятность противоположного события)
data_grouped_children.head(10)

debt       with_debt without_debt
          count   sum                       
children                                    
0         14091  1063  0.075438     0.924562
1          4808   444  0.092346     0.907654
2          2052   194  0.094542     0.905458
3           330    27  0.081818     0.918182
4            41     4  0.097561     0.902439
5             9     0  0.000000     1.000000

**Промежуточные наблюдения:**
* количество заемщиков в категории с 5 детьми всего лишь 9 человек, а в категории с 4 детьми лишь 41 человекпоэтому сделать объективный (статистический) вывод по такой небольшой выборке не получится. Для того чтобы не потерять эти данные - можно объединить заемщиков с 3 и более детьми в категорию "многодетные". Сделаем это следующим шагом
* по остальным категориям процент возврата кредита лежит в диапазоне от 90.55% до 92.46%. Максимальные процент возврата кредитов без задолженностей у категории без детей, а минимальный процент у категории с 2 детьми

Создадим в исходном датасете дополнительный стобец 'children_category' с категориями:
- без детей
- 1 ребенок
- 2 ребенка
- многодетный

Для этого создадим категоризирующую функцию children_categorize_func
    

In [ ]:
def children_categorize_func(ch):

    try:
        if ch == 0:
            return 'без детей'
        elif ch == 1:
            return '1 ребенок'
        elif ch == 2:
            return '2 ребенка'
        elif ch >=3:
            return 'многодетный'
        
    except:
        return 'не возможно присвоить категорию. Проверьте данные в столбце "children"'
        

Проверим работу новой функции:

In [ ]:
data['children_category'] = data['children'].apply(children_categorize_func)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,children_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,1 ребенок
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,1 ребенок
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,без детей
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,многодетный
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,без детей


In [ ]:
data['children_category'].unique()

array(['1 ребенок', 'без детей', 'многодетный', '2 ребенка'], dtype=object)

Видим, что функция верно присваивает значения и ошибок нет. Теперь повторим операцию по построению таблицы как было выполнено ранее

In [ ]:
data_grouped_children = data.groupby('children_category').agg({'debt' : ['count', 'sum']}) #группируем по новому столбцу
data_grouped_children['with_debt'] = data_grouped_children['debt']['sum'] / data_grouped_children['debt']['count']
data_grouped_children['without_debt'] = 1 - data_grouped_children['with_debt']
data_grouped_children.head(10)

debt       with_debt without_debt
                   count   sum                       
children_category                                    
1 ребенок           4808   444  0.092346     0.907654
2 ребенка           2052   194  0.094542     0.905458
без детей          14091  1063  0.075438     0.924562
многодетный          380    31  0.081579     0.918421

Для наглядности - отсортируем данные по убыванию вероятности вернуть кредит без задолженности и найдем самых надежных и ненадежных заемщиков по признаку наличия детей

In [ ]:
data_grouped_children.sort_values('without_debt', ascending=False)

debt       with_debt without_debt
                   count   sum                       
children_category                                    
без детей          14091  1063  0.075438     0.924562
многодетный          380    31  0.081579     0.918421
1 ребенок           4808   444  0.092346     0.907654
2 ребенка           2052   194  0.094542     0.905458

**Вывод:**
* Опираясь на полученные результаты видим, что при категоризации заемщиков по признаку 'количество детей' самыми надежными заемщика (с наибольшей вероятростью вернуть кредит без просрочек) являются заемщики без детей (вероятность 92.46%) и многодетные (вероятность 91.8%), а самыми ненадежными - с 1 ребенком (90.77%) и с 2 детьми (90.55%)
* Объянить такой результат можно тем, что возожно финансовое положение людей без детей чуть лучше (меньшие расходы), чем с 1 или с 2 детьми. А многодетные семьи могут получать субсидированные процентные ставки и другие льготы, а также если у людей 3 и более детей, то опять же с финансовой точки зрения - они могут себе это позволить и обеспечить такую большую семью

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

Краткое описание методики решения: 
* Определим отношение (процент) погашенных в срок кредитов к общему числу кредитов для каждой категории 'family_status';
* Построим график (гистограмму), на котором по оси Y укажем параметр 'вероятность возврата кредита', а по оси Х - категорию, или соберем данные в сводную таблицу, проведя сортировку по убыванию или возрастанию;
* Опираясь на результаты - сфорируем выводы

In [ ]:
data['family_status'].unique() # Посмотрим уникальные значения

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Дублирующихся значений нет (например, из-за ошибок шрифта/опечаток). Можно групировать данные по признаку 'family_status' и выполнить необходимы операции для расчета искомой вероятности возврата кредита без задолженности (столбец 'without_debt')

In [ ]:
data_grouped_family = data.groupby('family_status').agg({'debt' : ['count', 'sum']}) # группируем
data_grouped_family['with_debt'] = data_grouped_family['debt']['sum'] / data_grouped_family['debt']['count'] # вероятность вернуть кредит с задержкой
data_grouped_family['without_debt'] = 1 - data_grouped_family['with_debt'] # вероятность вернуть кредит без задержки (как противоположное событие)
data_grouped_family.head(10)

debt      with_debt without_debt
                       count  sum                       
family_status                                           
Не женат / не замужем   2796  273  0.097639     0.902361
в разводе               1189   84  0.070648     0.929352
вдовец / вдова           951   63  0.066246     0.933754
гражданский брак        4134  385  0.093130     0.906870
женат / замужем        12261  927  0.075606     0.924394

**Промежуточные наблюдения:**
* количество заемщиков в каждой достаточно велико (больше 950), что позволяет делать статистические выводы
* максимальная вероятность (93.38%) приходится на категорию "вдовец / вдова", а минимальная вероятность (90.24%) на категорию "Не женат / не замужем".

Для наглядности - отсортируем данные по убыванию вероятности вернуть кредит без задолженности и найдем самых надежных и ненадежных заемщиков по признаку семейный статус

In [ ]:
data_grouped_family.sort_values('without_debt', ascending=False)

debt      with_debt without_debt
                       count  sum                       
family_status                                           
вдовец / вдова           951   63  0.066246     0.933754
в разводе               1189   84  0.070648     0.929352
женат / замужем        12261  927  0.075606     0.924394
гражданский брак        4134  385  0.093130     0.906870
Не женат / не замужем   2796  273  0.097639     0.902361

**Вывод:** 
* Опираясь на полученные результаты видим, что при категоризации заемщиков по признаку 'семейный статус' самыми надежными заемщика (с наибольшей вероятростью вернуть кредит без просрочек) являются заемщики "вдовец / вдова" (вероятность 93.38%) и 'в разводе' (вероятность 92.93%), а самыми ненадежными - с 'Не женат / не замужем' (90.24%) и с 'гражданский брак' (90.68%)



#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

В предыдущих заданиях мы разбили всех заемщиков по категориям дохода:

- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'.

Краткое описание методики решения: 
* Определим отношение (процент) погашенных в срок кредитов к общему числу кредитов для каждой категории 'total_income_category';
* Построим график (гистограмму), на котором по оси Y укажем параметр 'вероятность возврата кредита', а по оси Х - категорию, или соберем данные в сводную таблицу, проведя сортировку по убыванию или возрастанию;
* Опираясь на результаты - сфорируем выводы

In [ ]:
data['total_income_category'].unique()

array(['B', 'C', 'D', 'E', 'A'], dtype=object)

Дублирующихся значений нет (например, из-за ошибок шрифта/опечаток). Можно групировать данные по признаку 'total_income_category' и выполнить необходимы операции для расчета искомой вероятности возврата кредита без задолженности (столбец 'without_debt')

In [ ]:
data_grouped_income = data.groupby('total_income_category').agg({'debt' : ['count', 'sum']}) # группируем
data_grouped_income['with_debt'] = data_grouped_income['debt']['sum'] / data_grouped_income['debt']['count'] # вероятность вернуть кредит с задержкой
data_grouped_income['without_debt'] = 1 - data_grouped_income['with_debt'] # вероятность вернуть кредит без задержки (как противоположное событие)
data_grouped_income.head(10)

debt       with_debt without_debt
                       count   sum                       
total_income_category                                    
A                         25     2  0.080000     0.920000
B                       5014   354  0.070602     0.929398
C                      15921  1353  0.084982     0.915018
D                        349    21  0.060172     0.939828
E                         22     2  0.090909     0.909091

**Промежуточные наблюдения:**
* для статистических выводов количество заемщиков слишком мало в категориях E (22 чел) и А (25 чел). Поэтому предлагается объединить смежные категории в более крупные: E и D, А и В. 

Для объединения категорий по доходу создадим функцию total_income_categ_new_func

In [ ]:
def total_income_categ_new_func(x):
    if x == 'A' or x == 'B':
        return 'A+B'
    elif x == 'C':
        return 'C'
    elif x == 'D' or x == 'E':
        return 'D+E'

In [ ]:
data['total_income_category_new'] = data['total_income_category'].apply(total_income_categ_new_func)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,children_category,total_income_category_new
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,1 ребенок,A+B
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,1 ребенок,C
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,без детей,C
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,многодетный,A+B
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,без детей,C


Таким образом ввели новый столбец с укрупненными категориями. Старый столбец затирать не стал - он может пригодиться в дальнейших исследованиях

In [ ]:
data_grouped_income = data.groupby('total_income_category_new').agg({'debt' : ['count', 'sum']}) # группируем
data_grouped_income['with_debt'] = data_grouped_income['debt']['sum'] / data_grouped_income['debt']['count'] # вероятность вернуть кредит с задержкой
data_grouped_income['without_debt'] = 1 - data_grouped_income['with_debt'] # вероятность вернуть кредит без задержки (как противоположное событие)
data_grouped_income.head(10)

debt       with_debt without_debt
                           count   sum                       
total_income_category_new                                    
A+B                         5039   356  0.070649     0.929351
C                          15921  1353  0.084982     0.915018
D+E                          371    23  0.061995     0.938005

Для наглядности - отсортируем данные по убыванию вероятности вернуть кредит без задолженности и найдем самых надежных и ненадежных заемщиков по признаку total income

In [ ]:
data_grouped_income.sort_values('without_debt', ascending=False)

debt       with_debt without_debt
                           count   sum                       
total_income_category_new                                    
D+E                          371    23  0.061995     0.938005
A+B                         5039   356  0.070649     0.929351
C                          15921  1353  0.084982     0.915018

**Промежуточные наблюдения:**

* Объединение категорий D и E привело к тому, что данная катгория оказалась самым надежным заемщиком. Однако, примем во внимание тот факт, что количество наблюдений по данной категории все еще мало (371). Примем, что для статистических выводов количество наблюдений должно быть больше 1000, поэтому результаты по группе D+E не будем учитывать в итогом выводе, указав на факт недостаточности количества наблюдений

**Вывод:** 
* Опираясь на полученные результаты видим, что при категоризации заемщиков по признаку 'total income' самыми надежными заемщика (с наибольшей вероятностью вернуть кредит без просрочек) являются заемщики с категорией дохода A+B (от 200 тыс) (вероятность 92.94%)  а самыми ненадежными - с категорией С (от 50 тыс до 200 тыс) (91.50%). Категории D+E исключили из рассмотрения 
* Результат, что самыми надежными заемщиками окажутся люди с высоким доходом был вполне ожидаем.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

Методика решения аналогична предыдущим пунктам.
Посмотрим уникальные значения в столбце 'purpose_category':

In [ ]:
data['purpose_category'].unique()# Ваш код будет здесь. Вы можете создавать новые ячейки.

array(['операции с недвижимостью', 'операции с автомобилем',
       'получение образования', 'проведение свадьбы'], dtype=object)

In [ ]:
data_grouped_purpose = data.groupby('purpose_category').agg({'debt' : ['count', 'sum']}) #группировка по столбцу 'purpose_category' и применение функций 
data_grouped_purpose['with_debt'] = data_grouped_purpose['debt']['sum'] / data_grouped_purpose['debt']['count'] # вероятность вернуть кредит с задержкой
data_grouped_purpose['without_debt'] = 1 - data_grouped_purpose['with_debt'] # вероятность вернуть кредит без задержки (как противоположное событие)

data_grouped_purpose.sort_values('without_debt', ascending=False) #сортировка по убыванию вероятности вернуть кредит без задержек


debt      with_debt without_debt
                          count  sum                       
purpose_category                                           
операции с недвижимостью  10751  780  0.072551     0.927449
проведение свадьбы         2313  183  0.079118     0.920882
получение образования      3988  369  0.092528     0.907472
операции с автомобилем     4279  400  0.093480     0.906520

**Промежуточные наблюдения**
* Количество наблюдений в каждой категории больше 1000, что позволяет делать статистические выводы

**Вывод:** 
* Опираясь на полученные результаты видим, что при категоризации заемщиков по признаку 'purpose_category' самыми надежными заемщика (с наибольшей вероятностью вернуть кредит без просрочек) являются заемщики, которые берут кредит на операции, связанные с недвижимостью (вероятность 92.74%)  а самыми ненадежными - заемщики, которые берут кредит на операции с автомобилем (90.65%). 
* Объяснить такой результат можно тем, что покупка недвижимости чаще чуть более продуманная и взвешенная покупка, чем покупка автомобиля которая нередко относится к эмоциональным покупкам. Для более точного понимания причины необходимо "копать глубже" - смотреть возраст заемщиков, категории дохода, образование и т.д.

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* Причины появления пропуска в данных могут быть различными:
* ошибка при выгрузке данных (следует пообщаться с разработчиком, который предоставил выгрузку и обсудить с ним пропуски в данных);
* человеческий фактор (неккоректное заполнение информации (здесь надо проработать формы (анкеты), которые заполняет пользователь и сделать поля, например, обязательными или с выбором из списка)) или намеренное сокртые информации);
* часть данных могля быть не заполнена, потому что она относится к разным категориям. Например, студент мог оставить пропуск в графе 'days_employed', так как он еще ни дня не работал.

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* Для количесвенных переменных лучше заполнять пропуски медианным значением, чтобы не получить искаженный результат при наличии сильных выбросов в данных (аномальных значений). 

Например, кейс - в фирме работает 10 человек, з/п у всех сотрудников - 100тыс. При приеме на работу сотрудника с з/п 300 тыс и пересчете средней з/п сотрудников путем нахождения среднего арифмеического мы получим существенный рост средней з/п (1300 тыс / 11 чел = 118 тыс, что соответсвует росту средней з/п всех сотрудников на 18%). И если з/п одного из сотрудников была бы пропущена, то взяв среднее арифметическое мы бы указали 118 тыс, а взяв медианное значение - 100 тыс, что ближе к реальной картине.

Также можно привести кейсы с недвижимостью (зависимость цены от площади), когда есть аномально высокие цены на объекты (с такой же площадью, но аномально высоким значением цены) которые "тянут среднее значение стоимости квадратного метра вверх". В этом случае, более правильным решением заоплнять пропуски в ценах - будет брать медианное значение стоимости площади кв метра.

### Шаг 4: общий вывод.

В процессе анализа данных были получены следующие выводы:

* При категоризации заемщиков по признаку 'количество детей' было применено укрупнение категорий и введение новой - "многодетные". Самыми надежными заемщика (с наибольшей вероятростью вернуть кредит без просрочек) являются заемщики без детей (вероятность 92.46%) и многодетные (вероятность 91.8%), а самыми ненадежными - с 1 ребенком (90.77%) и с 2 детьми (90.55%). Объянить такой результат можно тем, что возможно финансовое положение людей без детей чуть лучше (меньшие расходы), чем с 1 или с 2 детьми. А многодетные семьи могут получать субсидированные процентные ставки и другие льготы, а также если у людей 3 и более детей, то опять же с финансовой точки зрения - они могут себе это позволить и обеспечить такую большую семью;
* При категоризации заемщиков по признаку 'семейный статус' самыми надежными заемщика (с наибольшей вероятростью вернуть кредит без просрочек) являются заемщики "вдовец / вдова" (вероятность 93.38%) и 'в разводе' (вероятность 92.93%), а самыми ненадежными - с 'Не женат / не замужем' (90.24%) и с 'гражданский брак' (90.68%)
* При категоризации заемщиков по признаку 'total income' самыми надежными заемщика (с наибольшей вероятностью вернуть кредит без просрочек) являются заемщики с категорией дохода A+B (от 200 тыс) (вероятность 92.94%)  а самыми ненадежными - с категорией С (от 50 тыс до 200 тыс) (91.50%). Категории D+E исключили из рассмотрения из-за малости количества наблюдений. Результат, что самыми надежными заемщиками окажутся люди с высоким доходом был вполне ожидаем;
* При категоризации заемщиков по признаку 'purpose_category' самыми надежными заемщика (с наибольшей вероятностью вернуть кредит без просрочек) являются заемщики, которые берут кредит на операции, связанные с недвижимостью (вероятность 92.74%)  а самыми ненадежными - заемщики, которые берут кредит на операции с автомобилем (90.65%). Объяснить такой результат можно тем, что покупка недвижимости чаще чуть более продуманная и взвешенная покупка, чем покупка автомобиля которая нередко относится к эмоциональным покупкам. Для более точного понимания причины необходимо "копать глубже" - смотреть возраст заемщиков, категории дохода, образование и т.д.
